# ============================================================================
# NOTEBOOK 01: MODEL LOADING & ARCHITECTURE ANALYSIS
# ============================================================================
# 
# Objetivo:
#   - Cargar modelo ResNet-18 pre-entrenado
#   - Explorar arquitectura de la red
#   - Implementar sistema de hooks para capturar activaciones
#   - Realizar predicciones de prueba
#
# Duración estimada: 2-3 horas
# ============================================================================

In [ ]:
import torch

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ CUDA version: {torch.version.cuda}")
    
    # Test rápido
    x = torch.randn(3, 3).cuda()
    print(f"✅ Tensor en GPU: {x.device}")
else:
    print("❌ CUDA aún no disponible")


In [ ]:
# ============================================================================
# CELDA 1: IMPORTS Y CONFIGURACIÓN INICIAL
# ============================================================================

import sys
import os
from pathlib import Path

# Agregar directorio raíz al path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Imports de librerías estándar
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

# Imports del proyecto
from src.models.model_loader import ModelLoader
from src.utils.image_loader import ImageLoader
from src.utils.hooks import ActivationHook

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Configuración de reproducibilidad
torch.manual_seed(42)
np.random.seed(42)

print("✅ Imports completados")
print(f"📁 Directorio del proyecto: {project_root}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"💻 CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    

In [ ]:
# ============================================================================
# CELDA 2: CARGAR MODELO RESNET-18 PRE-ENTRENADO
# ============================================================================

print("\n" + "="*60)
print("🧠 CARGANDO MODELO RESNET-18")
print("="*60)

# Inicializar cargador de modelo
model_loader = ModelLoader(
    model_name='resnet18',
    pretrained=True,
    num_classes=1000  # ImageNet tiene 1000 clases
)

# Cargar modelo
model = model_loader.load_model()

print(f"\n✅ Modelo cargado exitosamente" )
print(f"📊 Tipo de modelo: {type(model).__name__}")
print(f"💾 Device: {model_loader.device}")
print(f"🎯 Número de clases: 1000 (ImageNet)")

In [ ]:
# ============================================================================
# CELDA 3: ANÁLISIS DE LA ARQUITECTURA DEL MODELO                                                          
# ============================================================================

print("\n" + "="*60)
print("🔍 ANÁLISIS DE ARQUITECTURA")
print("="*60)

# Obtener información de la arquitectura (helper method)
arch_info = model_loader.get_architecture_info()

print(f"\n📌 RESUMEN GENERAL")
print(f"{'='*50}")
print(f"Total de parámetros:      {arch_info['total_params']:,}")
print(f"Parámetros entrenables:   {arch_info['trainable_params']:,}") # Parámetros son los pesos (weights) y sesgos (biases)
print(f"Parámetros congelados:    {arch_info['frozen_params']:,}")
print(f"Tamaño del modelo:        {arch_info['model_size_mb']:.2f} MB")
print(f"Número de capas:          {arch_info['num_layers']}")

print(f"\n📊 DISTRIBUCIÓN DE PARÁMETROS POR TIPO")
print(f"{'='*50}")
for param_type, count in arch_info['params_by_type'].items():
    print(f"{param_type:20s}: {count:>12,} ({count/arch_info['total_params']*100:>5.1f}%)")

In [ ]:
# ============================================================================
# CELDA EXTRA: ENTENDER PARÁMETROS ENTRENABLES VS CONGELADOS
# ============================================================================

import torch
import torch.nn as nn
import torchvision.models as models

print("🔬 ANÁLISIS DE PARÁMETROS\n")
print("="*70)

# 1. Crear modelo simple
model = models.resnet18(pretrained=True)

# Función helper para contar parámetros
def count_parameters(model):
    total = 0
    trainable = 0
    frozen = 0
    
    for param in model.parameters():
        num_params = param.numel()
        total += num_params
        
        if param.requires_grad:
            trainable += num_params
        else:
            frozen += num_params
    
    return total, trainable, frozen

# 2. Estado inicial (todos entrenables)
total, trainable, frozen = count_parameters(model)
print("\n📊 ESTADO INICIAL (modelo pre-entrenado)")
print(f"Total parámetros:      {total:,}")
print(f"Entrenables:           {trainable:,} ({trainable/total*100:.1f}%)")
print(f"Congelados:            {frozen:,} ({frozen/total*100:.1f}%)")

# 3. Congelar TODAS las capas
print("\n❄️  CONGELANDO TODAS LAS CAPAS...")
for param in model.parameters():
    param.requires_grad = False

total, trainable, frozen = count_parameters(model)
print(f"Total parámetros:      {total:,}")
print(f"Entrenables:           {trainable:,} ({trainable/total*100:.1f}%)")
print(f"Congelados:            {frozen:,} ({frozen/total*100:.1f}%)")

# 4. Descongelar SOLO la última capa (fc)
print("\n🔥 DESCONGELANDO SOLO LA CAPA FC...")
for param in model.fc.parameters():
    param.requires_grad = True

total, trainable, frozen = count_parameters(model)
print(f"Total parámetros:      {total:,}")
print(f"Entrenables:           {trainable:,} ({trainable/total*100:.1f}%)")
print(f"Congelados:            {frozen:,} ({frozen/total*100:.1f}%)")

# 5. Ver detalles de cada capa
print("\n🔍 DETALLE POR CAPA (primeras 10):")
print("-"*70)
print(f"{'Capa':<30} {'Parámetros':<15} {'Entrenable'}")
print("-"*70)

for idx, (name, param) in enumerate(model.named_parameters()):
    if idx >= 10:  # Solo primeras 10
        print("...")
        break
    
    num_params = param.numel()
    trainable_mark = "✓" if param.requires_grad else "✗"
    print(f"{name:<30} {num_params:>12,}   {trainable_mark}")

# 6. Ver última capa
print("\n🎯 ÚLTIMA CAPA (fc):")
print("-"*70)
for name, param in model.fc.named_parameters():
    num_params = param.numel()
    trainable_mark = "✓" if param.requires_grad else "✗"
    print(f"{name:<30} {num_params:>12,}   {trainable_mark}")

print("\n" + "="*70)
print("✅ Análisis completado\n")

# 7. Ejemplo práctico: ¿Qué pasa durante el entrenamiento?
print("💡 EJEMPLO PRÁCTICO:")
print("-"*70)
print("""
Con la configuración actual:
- Las capas conv1, layer1, layer2, etc. NO se actualizarán
- Solo la capa 'fc' aprenderá durante el entrenamiento
- Esto es útil para:
  * Fine-tuning rápido en un nuevo dataset
  * Transfer learning (usar features de ImageNet)
  * Evitar overfitting con datasets pequeños
""")

In [ ]:
# ============================================================================
# CELDA 4: VISUALIZAR ESTRUCTURA DE CAPAS
# ============================================================================

print("\n" + "="*60)
print("🏗️ ESTRUCTURA DE CAPAS DE RESNET-18")
print("="*60)

# Obtener lista de capas con detalles
layers_info = model_loader.get_layers_info()

# FORMATO MEJORADO con columna Trainable
print(f"\n{'Capa':<30} {'Tipo':<25} {'Output Shape':<20} {'Parámetros':<15} {'Trainable'}")
print("-"*110)

for layer_info in layers_info[:15]:  # Mostrar primeras 15 capas
    # Símbolo visual para trainable
    trainable_symbol = "✓" if layer_info['trainable'] else "✗"
    
    print(f"{layer_info['name']:<30} "
          f"{layer_info['type']:<25} "
          f"{str(layer_info['output_shape']):<20} "
          f"{layer_info['params']:>12,}   "
          f"{trainable_symbol:>3}")

print(f"\n... (mostrando primeras 15 capas de {len(layers_info)})")
print(f"\n💡 Usar model_loader.get_layers_info() para ver todas las capas")

In [ ]:
# ============================================================================
# CELDA DE DIAGNÓSTICO: Investigar el error
# ============================================================================

print("\n🔍 DIAGNÓSTICO: Top 10 capas por parámetros")
print("="*80)

# Obtener info de capas
layers_info = model_loader.get_layers_info()

# Ordenar por parámetros
top_layers = sorted(layers_info, key=lambda x: x['params'], reverse=True)[:15]

# Mostrar información COMPLETA (sin truncar nombres)
print(f"\n{'#':<4} {'Nombre COMPLETO':<40} {'Tipo':<20} {'Parámetros':<15}")
print("-"*80)

for i, layer in enumerate(top_layers, 1):
    print(f"{i:<4} {layer['name']:<40} {layer['type']:<20} {layer['params']:>12,}")

print("\n" + "="*80)
print("💡 Observa si hay nombres duplicados o extraños")

In [ ]:
# ============================================================================
# CELDA 5: VISUALIZACIÓN GRÁFICA DE LA ARQUITECTURA
# ============================================================================

print("\n📊 VISUALIZACIÓN DE LA ARQUITECTURA")
print("="*60)

# Crear visualización de la arquitectura
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# --------------------------------------------------------
# SUBPLOT 1: Parámetros por capa (Top 10)
# --------------------------------------------------------
ax1 = axes[0]

# Ordenar por parámetros (solo capas con params > 0)
top_layers = sorted(
    [l for l in layers_info if l['params'] > 0], 
    key=lambda x: x['params'], 
    reverse=True
)[:10]

# Extraer nombres más descriptivos (mantener contexto)
layer_names = []
for layer in top_layers:
    full_name = layer['name']
    
    # Si el nombre tiene puntos, mantener primero y último
    if '.' in full_name:
        parts = full_name.split('.')
        if len(parts) >= 3:
            # Ejemplo: 'layer4.0.conv2' → 'L4.0.conv2'
            short_name = f"L{parts[0][-1]}.{parts[1]}.{parts[-1]}"
        else:
            # Ejemplo: 'layer4.downsample' → 'layer4.downs'
            short_name = '.'.join(parts[:2])[:15]
    else:
        # Sin puntos, usar tal cual
        short_name = full_name
    
    layer_names.append(short_name)

layer_params = [l['params'] for l in top_layers]

# Crear barras horizontales
bars1 = ax1.barh(layer_names, layer_params, color='skyblue', edgecolor='black')

ax1.set_xlabel('Número de Parámetros', fontsize=11)
ax1.set_title('Top 10 Capas por Parámetros', fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Agregar valores en las barras (formato compacto)
for i, (bar, val) in enumerate(zip(bars1, layer_params)):
    if val > 0:
        # Formato en millones o miles
        if val >= 1e6:
            label = f' {val/1e6:.1f}M'
        elif val >= 1e3:
            label = f' {val/1e3:.0f}K'
        else:
            label = f' {val:,}'
        
        ax1.text(val, i, label, va='center', fontsize=9)

# --------------------------------------------------------
# SUBPLOT 2: Distribución de tipos de capas
# --------------------------------------------------------
ax2 = axes[1]

# PASO 1: Contar cuántas capas hay de cada tipo
layer_types = {}
for layer in layers_info:
    layer_type = layer['type'] # 'Conv2d', 'BatchNorm2d', etc.
    layer_types[layer_type] = layer_types.get(layer_type, 0) + 1

# Filtrar tipos con menos de 2 capas para claridad (ajustable)
filtered_types = {k: v for k, v in layer_types.items() if v >= 2}
filtered_types['Otros'] = sum(v for k, v in layer_types.items() if v < 2)

colors = plt.cm.Set3(range(len(filtered_types)))
wedges, texts, autotexts = ax2.pie(
    filtered_types.values(), 
    labels=filtered_types.keys(),
    autopct='%1.1f%%',
    colors=colors,
    startangle=90
)
ax2.set_title('Distribución de Tipos de Capas', fontsize=12, fontweight='bold')

for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontsize(9)
    autotext.set_fontweight('bold')

# --------------------------------------------------------
# SUBPLOT 3: Parámetros acumulados por capa
# --------------------------------------------------------
ax3 = axes[2]
cumulative_params = []
current_sum = 0
x_positions = []

for i, layer in enumerate(layers_info):
    current_sum += layer['params']
    cumulative_params.append(current_sum)
    x_positions.append(i)

ax3.plot(x_positions, cumulative_params, linewidth=2, color='coral')
ax3.fill_between(x_positions, cumulative_params, alpha=0.3, color='coral')
ax3.set_xlabel('Índice de Capa', fontsize=11)
ax3.set_ylabel('Parámetros Acumulados', fontsize=11)
ax3.set_title('Parámetros Acumulados por Profundidad', fontsize=12, fontweight='bold')
ax3.grid(alpha=0.3)

# Formato del eje Y
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M'))

plt.tight_layout()
plt.show()

print("\n✅ Visualizaciones generadas correctamente")

In [ ]:
# ============================================================================
# CELDA 6: ESTRUCTURA ESPECÍFICA DE RESNET-18
# ============================================================================

print("\n" + "="*60)
print("🔬 BLOQUES RESIDUALES DE RESNET-18")
print("="*60)

# Analizar estructura de bloques residuales
residual_blocks = model_loader.get_residual_blocks_info()

print(f"\n📦 ResNet-18 tiene {len(residual_blocks)} bloques residuales:")
print(f"{'='*70}")
# Por la arquitectura de ResNet-18, el primer bloque no es residual por lo que el 0 y 1 son iguales
for i, block_info in enumerate(residual_blocks[1:], 1): 
    print(f"\n🔹 BLOQUE {i}: {block_info['name']}")
    print(f"   Capas: {block_info['num_layers']}")
    print(f"   Parámetros: {block_info['params']:,}")
    print(f"   Input channels: {block_info['in_channels']}")
    print(f"   Output channels: {block_info['out_channels']}")
    print(f"   Stride: {block_info['stride']}")
    print(f"   Tiene shortcut: {block_info['has_downsample']}")

print(f"\n{'='*70}")
print(f"💡 Los bloques residuales permiten el flujo de gradientes en redes profundas")

In [ ]:
# ============================================================================
# CELDA 7: CARGAR DATASET CIFAR-10 PARA TESTING
# ============================================================================

print("\n" + "="*60)
print("📦 CARGANDO DATASET CIFAR-10")
print("="*60)

# Inicializar cargador de imágenes
image_loader = ImageLoader(
    dataset_name='cifar10',
    batch_size=32,
    num_workers=2
)

# Cargar datasets
train_loader, test_loader = image_loader.get_dataloaders()

# Información del dataset
dataset_info = image_loader.get_dataset_info()

print(f"\n✅ Dataset cargado exitosamente")
print(f"{'='*50}")
print(f"Dataset: {dataset_info['name']}")
print(f"Número de clases: {dataset_info['num_classes']}")
print(f"Tamaño de imágenes: {dataset_info['image_size']}")
print(f"Muestras de entrenamiento: {dataset_info['train_samples']:,}")
print(f"Muestras de prueba: {dataset_info['test_samples']:,}")

print(f"\n📊 CLASES DE CIFAR-10:")
print(f"{'='*50}")
for idx, class_name in enumerate(dataset_info['classes']):
    print(f"{idx}: {class_name}")

In [ ]:
# ============================================================================
# CELDA 8: VISUALIZAR MUESTRAS DEL DATASET
# ============================================================================

print("\n📸 VISUALIZANDO MUESTRAS DE CIFAR-10")
print("="*60)

# Obtener un batch de imágenes
images, labels = next(iter(test_loader))

# Visualizar 16 imágenes
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
fig.suptitle('Muestras de CIFAR-10 (Test Set)', fontsize=16, fontweight='bold')

class_names = dataset_info['classes']

for idx, ax in enumerate(axes.flat):
    if idx < len(images):
        # Denormalizar imagen para visualización
        img = image_loader.denormalize_image(images[idx])
        
        ax.imshow(img)
        ax.set_title(f'{class_names[labels[idx]]}', fontsize=10)
        ax.axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ Visualización completada")
print(f"💡 Nota: Las imágenes fueron normalizadas con estadísticas de ImageNet")

In [ ]:
# ============================================================================
# CELDA 8.5: AJUSTAR MODELO PARA CIFAR-10
# ============================================================================

print("\n" + "="*60)
print("🔧 AJUSTANDO MODELO PARA CIFAR-10")
print("="*60)

# El modelo ResNet está pre-entrenado en ImageNet (1000 clases)
# Pero vamos a usar CIFAR-10 (10 clases)
# Necesitamos modificar la última capa

print(f"\n📊 Configuración original:")
print(f"   Capa fc: {model.fc}")

# Obtener features de entrada de la última capa
num_features = model.fc.in_features

# Reemplazar última capa
model.fc = nn.Linear(num_features, 10)  # 10 clases de CIFAR-10

# Mover a device
model = model.to(model_loader.device)
model.eval()

print(f"\n✅ Modelo ajustado:")
print(f"   Capa fc: {model.fc}")
print(f"   Ahora el modelo predice {10} clases (CIFAR-10)")

print(f"\n⚠️ NOTA: Los pesos de fc son aleatorios (no pre-entrenados)")
print(f"   El accuracy será bajo (~10%) hasta que entrenes el modelo")
print(f"   Pero las predicciones estarán en el rango correcto [0-9]")

In [ ]:
# ============================================================================
# CELDA 9: AJUSTAR MODELO Y REALIZAR PREDICCIONES DE PRUEBA
# ============================================================================

print("\n" + "="*60)
print("🔧 AJUSTANDO MODELO PARA CIFAR-10")
print("="*60)

# El modelo ResNet está pre-entrenado en ImageNet (1000 clases)
# Pero vamos a usar CIFAR-10 (10 clases)
# Necesitamos modificar la última capa

print(f"\n📊 Configuración original:")
print(f"   Última capa: Linear(in_features={model.fc.in_features}, out_features={model.fc.out_features})")

# Obtener features de entrada de la última capa
num_features = model.fc.in_features

# Reemplazar última capa para 10 clases
model.fc = nn.Linear(num_features, 10)  # 10 clases de CIFAR-10

# Mover a device y configurar en modo evaluación
model = model.to(model_loader.device)
model.eval()

print(f"\n✅ Modelo ajustado:")
print(f"   Nueva capa fc: Linear(in_features={num_features}, out_features=10)")
print(f"   Ahora el modelo predice 10 clases (CIFAR-10)")

print(f"\n⚠️ NOTA: Los pesos de fc son aleatorios (no pre-entrenados)")
print(f"   El accuracy será bajo (~10%) hasta que se entrene el modelo")

# ============================================================================
# REALIZAR PREDICCIONES
# ============================================================================

print("\n" + "="*60)
print("🎯 REALIZANDO PREDICCIONES CON EL MODELO")
print("="*60)

# Realizar predicciones en el batch de prueba
with torch.no_grad():
    outputs = model(images.to(model_loader.device))
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    confidences, predictions = torch.max(probabilities, 1)

# Mover a CPU para procesamiento
predictions = predictions.cpu().numpy()
confidences = confidences.cpu().numpy()
labels_np = labels.numpy()

# Calcular accuracy en este batch
correct = (predictions == labels_np).sum()
accuracy = correct / len(labels_np) * 100

print(f"\n📊 RESULTADOS EN BATCH DE PRUEBA")
print(f"{'='*50}")
print(f"Tamaño del batch: {len(labels_np)}")
print(f"Predicciones correctas: {correct}/{len(labels_np)}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Confianza promedio: {confidences.mean():.4f}")

print(f"\n🔍 EJEMPLOS DE PREDICCIONES:")
print(f"{'='*60}")
print(f"{'Real':<15} {'Predicción':<15} {'Confianza':<12} {'Correcto'}")
print("-"*60)

for i in range(min(10, len(labels_np))):
    real_class = class_names[labels_np[i]]
    pred_class = class_names[predictions[i]]
    conf = confidences[i]
    correct_mark = "✓" if labels_np[i] == predictions[i] else "✗"
    
    print(f"{real_class:<15} {pred_class:<15} {conf:>6.2%}       {correct_mark}")

print(f"\n{'='*60}")
print(f"💡 Accuracy bajo es esperado porque la capa fc tiene pesos aleatorios")
print(f"   En notebooks futuros entrenaremos el modelo para mejorar el accuracy")

In [ ]:
# ============================================================================
# CELDA 10: IMPLEMENTAR HOOKS PARA CAPTURAR ACTIVACIONES
# ============================================================================
# como se modifica la ultima capa, nos nos preocupa la prediccion correcta
# Sino que buscamos interpretar las activaciones internas de cada capa

print("\n" + "="*60)
print("🪝 CONFIGURANDO HOOKS PARA CAPTURAR ACTIVACIONES")
print("="*60)

# Definir capas de interés para capturar activaciones
target_layers = [
    'conv1',           # Primera capa convolucional
    'layer1.0.conv1',  # Primer bloque residual
    'layer2.0.conv1',  # Segundo bloque residual
    'layer3.0.conv1',  # Tercer bloque residual
    'layer4.0.conv1',  # Cuarto bloque residual
    'avgpool',         # Global average pooling
    'fc'               # Capa fully connected final
]

# Inicializar sistema de hooks
activation_hook = ActivationHook(model, target_layers)

# Registrar hooks
activation_hook.register_hooks()

print(f"\n✅ Hooks registrados en {len(target_layers)} capas")
print(f"{'='*50}")
print("Capas con hooks:")
for layer_name in target_layers:
    print(f"  • {layer_name}")

print(f"\n💡 Los hooks capturarán automáticamente las activaciones durante el forward pass")

In [ ]:
# ============================================================================
# CELDA 11: CAPTURAR Y ANALIZAR ACTIVACIONES DE UNA IMAGEN
# ============================================================================

print("\n" + "="*60)
print("📊 CAPTURANDO ACTIVACIONES DE UNA IMAGEN")
print("="*60)

# Tomar una imagen del batch
test_image = images[0:1].to(model_loader.device)  # Batch size = 1
test_label = labels[0].item()

print(f"\n🖼️ Imagen de prueba: {class_names[test_label]}")

# Forward pass para capturar activaciones
with torch.no_grad():
    output = model(test_image)
    prediction = torch.argmax(output, dim=1).item()
    confidence = torch.nn.functional.softmax(output, dim=1)[0, prediction].item()

print(f"Predicción: {class_names[prediction]}")
print(f"Confianza: {confidence:.2%}")

# Obtener activaciones capturadas
activations = activation_hook.get_activations()

print(f"\n📦 ACTIVACIONES CAPTURADAS:")
print(f"{'='*70}")
print(f"{'Capa':<30} {'Shape':<25} {'Min':<10} {'Max':<10} {'Mean'}")
print("-"*70)

for layer_name, activation in activations.items():
    shape = tuple(activation.shape)
    min_val = activation.min().item()
    max_val = activation.max().item()
    mean_val = activation.mean().item()
    
    print(f"{layer_name:<30} {str(shape):<25} "
          f"{min_val:>8.3f}  {max_val:>8.3f}  {mean_val:>8.3f}")

# Limpiar activaciones para la siguiente imagen
activation_hook.clear_activations()

In [ ]:
# ============================================================================
# CELDA 12: VISUALIZAR ACTIVACIONES DE CONV1 CON INTERPRETACIÓN
# ============================================================================

print("\n" + "="*60)
print("🎨 VISUALIZANDO ACTIVACIONES DE CONV1")
print("="*60)

# Hacer forward pass nuevamente
with torch.no_grad():
    output = model(test_image)

# Obtener activaciones de conv1
activations = activation_hook.get_activations()
conv1_activation = activations['conv1'].cpu().squeeze()  # Shape: [64, H, W]

print(f"Shape de activación conv1: {conv1_activation.shape}")
print(f"Número de filtros: {conv1_activation.shape[0]}")

# ============================================================================
# FUNCIÓN AUXILIAR: Interpretar qué detecta cada filtro
# ============================================================================

def interpretar_filtro(activation_map, filtro_idx):
    """
    Intenta inferir qué tipo de patrón detecta un filtro basándose en sus activaciones.
    
    Args:
        activation_map: Mapa de activación [H, W]
        filtro_idx: Índice del filtro
    
    Returns:
        Descripción del tipo de filtro
    """
    # Convertir a numpy
    act = activation_map.numpy()
    
    # Calcular estadísticas
    mean_val = act.mean()
    std_val = act.std()
    max_val = act.max()
    sparsity = (act == 0).sum() / act.size  # % de valores cero
    
    # Calcular gradientes (cambios bruscos = bordes)
    grad_y = np.abs(np.diff(act, axis=0)).mean()
    grad_x = np.abs(np.diff(act, axis=1)).mean()
    avg_gradient = (grad_y + grad_x) / 2
    
    # Clasificar según patrones
    descripcion = []
    
    # 1. Detectores de bordes (alto gradiente)
    if avg_gradient > 2.0:
        if grad_y > grad_x * 1.5:
            descripcion.append("🔲 Bordes Horizontales")
        elif grad_x > grad_y * 1.5:
            descripcion.append("🔳 Bordes Verticales")
        else:
            descripcion.append("◼️ Bordes Generales")
    
    # 2. Detectores de textura (variabilidad media-alta)
    elif std_val > 1.5 and avg_gradient > 0.5:
        descripcion.append("🌀 Textura/Patrón")
    
    # 3. Detectores de color/intensidad (bajo gradiente, activación uniforme)
    elif std_val < 1.0 and mean_val > 1.0:
        descripcion.append("🎨 Región Uniforme/Color")
    
    # 4. Filtro inactivo/muerto
    elif sparsity > 0.8 or max_val < 0.5:
        descripcion.append("💤 Inactivo")
    
    # 5. Alta activación = región importante
    elif max_val > 5.0:
        descripcion.append("⭐ Alta Activación")
    
    # 6. Default
    else:
        descripcion.append("🔍 Detector General")
    
    # Agregar métricas
    descripcion.append(f"(max:{max_val:.1f})")
    
    return " ".join(descripcion)

# ============================================================================
# VISUALIZAR PRIMEROS 16 FILTROS CON INTERPRETACIÓN
# ============================================================================

fig, axes = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle(f'Activaciones de Conv1 - Imagen: {class_names[test_label]}', 
             fontsize=16, fontweight='bold', y=0.995)

for idx, ax in enumerate(axes.flat):
    if idx < conv1_activation.shape[0]:
        activation_map = conv1_activation[idx]
        
        # Interpretar qué detecta este filtro
        interpretacion = interpretar_filtro(activation_map, idx)
        
        # Visualizar
        im = ax.imshow(activation_map.numpy(), cmap='viridis')
        
        # Título con número de filtro e interpretación
        ax.set_title(f'Filtro {idx}\n{interpretacion}', 
                     fontsize=9, pad=8)
        ax.axis('off')
        
        # Colorbar
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

# ============================================================================
# ESTADÍSTICAS DE TIPOS DE FILTROS
# ============================================================================

print(f"\n📊 ANÁLISIS DE FILTROS EN CONV1:")
print(f"{'='*60}")

# Clasificar todos los filtros
tipos_filtros = {
    'Bordes': 0,
    'Textura': 0,
    'Color/Uniforme': 0,
    'Inactivos': 0,
    'Alta Activación': 0,
    'General': 0
}

for idx in range(conv1_activation.shape[0]):
    interpretacion = interpretar_filtro(conv1_activation[idx], idx)
    
    if 'Bordes' in interpretacion:
        tipos_filtros['Bordes'] += 1
    elif 'Textura' in interpretacion:
        tipos_filtros['Textura'] += 1
    elif 'Uniforme' in interpretacion or 'Color' in interpretacion:
        tipos_filtros['Color/Uniforme'] += 1
    elif 'Inactivo' in interpretacion:
        tipos_filtros['Inactivos'] += 1
    elif 'Alta Activación' in interpretacion:
        tipos_filtros['Alta Activación'] += 1
    else:
        tipos_filtros['General'] += 1

print(f"\nDistribución de tipos de filtros:")
print(f"{'-'*60}")
for tipo, count in tipos_filtros.items():
    porcentaje = (count / conv1_activation.shape[0]) * 100
    barra = '█' * int(porcentaje / 5)  # Barra visual
    print(f"{tipo:<20} {count:>3} ({porcentaje:>5.1f}%) {barra}")

print(f"\n✅ Visualización completada")
print(f"💡 Conv1 detecta patrones básicos: bordes, texturas y colores")
print(f"💡 Capas más profundas combinan estos patrones para detectar objetos")

In [ ]:
# ============================================================================
# CELDA 13: ESTADÍSTICAS DE ACTIVACIONES POR CAPA
# ============================================================================

print("\n" + "="*60)
print("📈 ESTADÍSTICAS DE ACTIVACIONES POR CAPA")
print("="*60)

# Calcular estadísticas completas
activation_stats = {}

for layer_name, activation in activations.items():
    act_flat = activation.cpu().flatten().numpy()
    
    stats = {
        'mean': np.mean(act_flat),
        'std': np.std(act_flat),
        'min': np.min(act_flat),
        'max': np.max(act_flat),
        'sparsity': (act_flat == 0).sum() / len(act_flat),  # % de ceros
        'active_neurons': (act_flat > 0).sum(),
        'total_neurons': len(act_flat)
    }
    activation_stats[layer_name] = stats

# Visualizar estadísticas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Estadísticas de Activaciones por Capa', fontsize=14, fontweight='bold')

layer_names = list(activation_stats.keys())

# --------------------------------------------------------
# SUBPLOT 1: Media de activaciones
# --------------------------------------------------------
ax1 = axes[0, 0]
means = [activation_stats[l]['mean'] for l in layer_names]
ax1.bar(range(len(layer_names)), means, color='skyblue', edgecolor='black')
ax1.set_xticks(range(len(layer_names)))
ax1.set_xticklabels([l.replace('.', '\n') for l in layer_names], rotation=0, fontsize=8)
ax1.set_ylabel('Media de Activación')
ax1.set_title('Media de Activaciones por Capa')
ax1.grid(axis='y', alpha=0.3)

# --------------------------------------------------------
# SUBPLOT 2: Desviación estándar
# --------------------------------------------------------
ax2 = axes[0, 1]
stds = [activation_stats[l]['std'] for l in layer_names]
ax2.bar(range(len(layer_names)), stds, color='coral', edgecolor='black')
ax2.set_xticks(range(len(layer_names)))
ax2.set_xticklabels([l.replace('.', '\n') for l in layer_names], rotation=0, fontsize=8)
ax2.set_ylabel('Desviación Estándar')
ax2.set_title('Variabilidad de Activaciones')
ax2.grid(axis='y', alpha=0.3)

# --------------------------------------------------------
# SUBPLOT 3: Sparsity (% de neuronas inactivas)
# --------------------------------------------------------
ax3 = axes[1, 0]
sparsities = [activation_stats[l]['sparsity']*100 for l in layer_names]
ax3.bar(range(len(layer_names)), sparsities, color='lightgreen', edgecolor='black')
ax3.set_xticks(range(len(layer_names)))
ax3.set_xticklabels([l.replace('.', '\n') for l in layer_names], rotation=0, fontsize=8)
ax3.set_ylabel('Sparsity (%)')
ax3.set_title('Porcentaje de Neuronas Inactivas (=0)')
ax3.grid(axis='y', alpha=0.3)

# --------------------------------------------------------
# SUBPLOT 4: Rango de activación (min-max)
# --------------------------------------------------------
ax4 = axes[1, 1]
x = range(len(layer_names))
mins = [activation_stats[l]['min'] for l in layer_names]
maxs = [activation_stats[l]['max'] for l in layer_names]

ax4.scatter(x, mins, color='blue', label='Min', s=100, alpha=0.7, edgecolor='black')
ax4.scatter(x, maxs, color='red', label='Max', s=100, alpha=0.7, edgecolor='black')
ax4.plot(x, mins, 'b--', alpha=0.3)
ax4.plot(x, maxs, 'r--', alpha=0.3)
ax4.fill_between(x, mins, maxs, alpha=0.2)
ax4.set_xticks(range(len(layer_names)))
ax4.set_xticklabels([l.replace('.', '\n') for l in layer_names], rotation=0, fontsize=8)
ax4.set_ylabel('Valor de Activación')
ax4.set_title('Rango de Activaciones (Min-Max)')
ax4.legend()
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Análisis estadístico completado")

In [ ]:
# ============================================================================
# CELDA 14: VISUALIZAR PESOS DE LOS FILTROS DE CONV1
# ============================================================================

print("\n" + "="*60)
print("🔬 VISUALIZANDO PESOS DE FILTROS DE CONV1")
print("="*60)

# Extraer pesos de la primera capa convolucional
conv1_weights = model.conv1.weight.data.cpu()

print(f"Shape de pesos conv1: {conv1_weights.shape}")
print(f"Formato: [out_channels, in_channels, kernel_height, kernel_width]")
print(f"Número de filtros: {conv1_weights.shape[0]}")
print(f"Canales de entrada: {conv1_weights.shape[1]} (RGB)")
print(f"Tamaño del kernel: {conv1_weights.shape[2]}x{conv1_weights.shape[3]}")

# Visualizar primeros 16 filtros
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
fig.suptitle('Pesos de los Filtros de Conv1 (7x7 kernels)', 
             fontsize=14, fontweight='bold')

for idx, ax in enumerate(axes.flat):
    if idx < conv1_weights.shape[0]:
        # Obtener filtro [3, 7, 7]
        filter_weights = conv1_weights[idx].numpy()
        
        # Normalizar para visualización (transponer a [7, 7, 3] para RGB)
        filter_viz = np.transpose(filter_weights, (1, 2, 0))
        
        # Normalizar entre 0 y 1
        filter_viz = (filter_viz - filter_viz.min()) / (filter_viz.max() - filter_viz.min())
        
        ax.imshow(filter_viz)
        ax.set_title(f'Filtro {idx}', fontsize=9)
        ax.axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ Visualización de pesos completada")
print(f"💡 Estos filtros se aprenden para detectar patrones básicos en la primera capa")

In [ ]:
# ============================================================================
# CELDA 15: RESUMEN DINÁMICO Y PRÓXIMOS PASOS
# ============================================================================

print("\n" + "="*80)
print("📋 RESUMEN DEL NOTEBOOK 01 - ANÁLISIS AUTOMÁTICO")
print("="*80)

# ============================================================================
# RECOPILAR INFORMACIÓN DE CELDAS ANTERIORES
# ============================================================================

# 1. Información del modelo
arch_info = model_loader.get_architecture_info()
layers_info = model_loader.get_layers_info()
residual_blocks = model_loader.get_residual_blocks_info()

# 2. Información del dataset
dataset_info = image_loader.get_dataset_info()

# 3. Información de activaciones capturadas
activation_stats = activation_hook.get_activation_statistics()

# 4. Resultados de predicción (de celda 9)
# Ya tenemos: accuracy, correct, len(labels_np), confidences

# 5. Análisis de filtros (si ejecutaste celda 12 mejorada)
# Calcular tipos de filtros
conv1_activation = activations['conv1'].cpu().squeeze()
tipos_filtros = {
    'Bordes': 0,
    'Textura': 0,
    'Color/Uniforme': 0,
    'Inactivos': 0,
    'Alta Activación': 0,
    'General': 0
}

def clasificar_filtro_simple(activation_map):
    """Clasificación simplificada de filtros"""
    act = activation_map.numpy()
    mean_val = act.mean()
    std_val = act.std()
    max_val = act.max()
    sparsity = (act == 0).sum() / act.size
    
    grad_y = np.abs(np.diff(act, axis=0)).mean() if act.shape[0] > 1 else 0
    grad_x = np.abs(np.diff(act, axis=1)).mean() if act.shape[1] > 1 else 0
    avg_gradient = (grad_y + grad_x) / 2
    
    if avg_gradient > 2.0:
        return 'Bordes'
    elif std_val > 1.5 and avg_gradient > 0.5:
        return 'Textura'
    elif std_val < 1.0 and mean_val > 1.0:
        return 'Color/Uniforme'
    elif sparsity > 0.8 or max_val < 0.5:
        return 'Inactivos'
    elif max_val > 5.0:
        return 'Alta Activación'
    else:
        return 'General'

for idx in range(conv1_activation.shape[0]):
    tipo = clasificar_filtro_simple(conv1_activation[idx])
    tipos_filtros[tipo] += 1

# ============================================================================
# SECCIÓN 1: LOGROS COMPLETADOS
# ============================================================================

print("\n✅ LOGROS COMPLETADOS:")
print("="*80)

print(f"""
1. ✓ Modelo {arch_info['model_name'].upper()} cargado exitosamente
   • Parámetros: {arch_info['total_params']:,}
   • Tamaño: {arch_info['model_size_mb']:.2f} MB
   • Bloques residuales: {len(residual_blocks)}

2. ✓ Dataset {dataset_info['name']} cargado
   • Muestras train: {dataset_info['train_samples']:,}
   • Muestras test: {dataset_info['test_samples']:,}
   • Clases: {dataset_info['num_classes']}

3. ✓ Sistema de hooks implementado
   • Capas monitoreadas: {len(activation_hook.get_layer_names())}
   • Activaciones capturadas: {len(activations)}

4. ✓ Modelo ajustado para CIFAR-10
   • Última capa modificada: 512 → {dataset_info['num_classes']} clases
   • Predicciones en rango correcto: [0-{dataset_info['num_classes']-1}]

5. ✓ Predicciones de prueba realizadas
   • Batch evaluado: {len(labels_np)} imágenes
   • Accuracy: {accuracy:.2f}%
   • Predicciones correctas: {correct}/{len(labels_np)}

6. ✓ Activaciones analizadas en {len(activation_stats)} capas
   • Estadísticas completas calculadas
   • Salud del modelo verificada

7. ✓ Filtros de conv1 visualizados y clasificados
   • Total de filtros: {conv1_activation.shape[0]}
   • Filtros de bordes: {tipos_filtros['Bordes']}
   • Filtros de textura: {tipos_filtros['Textura']}
   • Filtros inactivos: {tipos_filtros['Inactivos']}
""")

# ============================================================================
# SECCIÓN 2: HALLAZGOS CLAVE (ANÁLISIS CRÍTICO)
# ============================================================================

print("="*80)
print("📊 HALLAZGOS CLAVE:")
print("="*80)

# Analizar confianza promedio
conf_mean = confidences.mean()
conf_std = confidences.std()

print(f"\n🎯 RENDIMIENTO DEL MODELO:")
if accuracy < 15:
    print(f"   ⚠️ Accuracy muy bajo ({accuracy:.2f}%) - ESPERADO")
    print(f"   • La última capa tiene pesos aleatorios (no entrenada)")
    print(f"   • Accuracy aleatorio para {dataset_info['num_classes']} clases: ~{100/dataset_info['num_classes']:.1f}%")
    print(f"   • Modelo actual: {accuracy:.2f}% (cerca del azar)")
elif accuracy < 25:
    print(f"   ⚠️ Accuracy bajo ({accuracy:.2f}%) pero mejor que azar")
    print(f"   • Las capas convolucionales pre-entrenadas ayudan un poco")
else:
    print(f"   ✅ Accuracy superior al azar ({accuracy:.2f}%)")
    print(f"   • Las features de ImageNet son útiles para CIFAR-10")

print(f"   • Confianza promedio: {conf_mean:.2%} (±{conf_std:.2%})")

if conf_mean < 0.15:
    print(f"   ⚠️ Modelo muy inseguro en sus predicciones")
elif conf_mean < 0.30:
    print(f"   ⚠️ Confianza baja (esperado con pesos aleatorios)")
else:
    print(f"   ✅ Confianza aceptable")

# Analizar activaciones
print(f"\n🔬 SALUD DE LAS ACTIVACIONES:")

problemas_detectados = []
capas_saludables = 0

for layer_name, stats in activation_stats.items():
    # Detectar problemas
    if stats['sparsity'] > 0.8:
        problemas_detectados.append(f"   ⚠️ {layer_name}: {stats['sparsity']*100:.1f}% sparsity (neuronas muertas)")
    elif stats['mean'] < -5 or stats['mean'] > 5:
        problemas_detectados.append(f"   ⚠️ {layer_name}: mean={stats['mean']:.2f} (valores extremos)")
    elif stats['std'] < 0.01:
        problemas_detectados.append(f"   ⚠️ {layer_name}: std={stats['std']:.4f} (sin variabilidad)")
    else:
        capas_saludables += 1

if len(problemas_detectados) == 0:
    print(f"   ✅ Todas las {len(activation_stats)} capas funcionan correctamente")
elif len(problemas_detectados) <= 2:
    print(f"   ✅ {capas_saludables}/{len(activation_stats)} capas saludables")
    print(f"   ⚠️ {len(problemas_detectados)} problema(s) menores detectado(s):")
    for problema in problemas_detectados:
        print(problema)
else:
    print(f"   ⚠️ {len(problemas_detectados)} problemas detectados:")
    for problema in problemas_detectados[:3]:  # Mostrar primeros 3
        print(problema)
    if len(problemas_detectados) > 3:
        print(f"   ... y {len(problemas_detectados)-3} más")

# Analizar distribución de filtros
print(f"\n🎨 DIVERSIDAD DE FILTROS EN CONV1:")
total_filtros = sum(tipos_filtros.values())
filtros_activos = total_filtros - tipos_filtros['Inactivos']

print(f"   • Filtros activos: {filtros_activos}/{total_filtros} ({filtros_activos/total_filtros*100:.1f}%)")

if tipos_filtros['Inactivos'] > total_filtros * 0.3:
    print(f"   ⚠️ {tipos_filtros['Inactivos']} filtros inactivos ({tipos_filtros['Inactivos']/total_filtros*100:.1f}%) - Alto")
elif tipos_filtros['Inactivos'] > 0:
    print(f"   ✅ Solo {tipos_filtros['Inactivos']} filtros inactivos - Aceptable")
else:
    print(f"   ✅ Todos los filtros activos - Excelente")

if tipos_filtros['Bordes'] > 0:
    print(f"   • Detectores de bordes: {tipos_filtros['Bordes']} ({tipos_filtros['Bordes']/total_filtros*100:.1f}%)")
if tipos_filtros['Textura'] > 0:
    print(f"   • Detectores de textura: {tipos_filtros['Textura']} ({tipos_filtros['Textura']/total_filtros*100:.1f}%)")

diversidad = len([v for v in tipos_filtros.values() if v > 0])
if diversidad >= 4:
    print(f"   ✅ Buena diversidad: {diversidad} tipos diferentes de filtros")
else:
    print(f"   ⚠️ Baja diversidad: solo {diversidad} tipos de filtros")

# Analizar estructura del modelo
print(f"\n🏗️ ARQUITECTURA DEL MODELO:")

# Encontrar capa más pesada
layers_with_params = [l for l in layers_info if l['params'] > 0]
if layers_with_params:
    capa_mas_pesada = max(layers_with_params, key=lambda x: x['params'])
    porcentaje_pesada = (capa_mas_pesada['params'] / arch_info['total_params']) * 100
    
    print(f"   • Capa más pesada: {capa_mas_pesada['name']}")
    print(f"     - Parámetros: {capa_mas_pesada['params']:,} ({porcentaje_pesada:.1f}% del total)")
    
    if porcentaje_pesada > 30:
        print(f"     ⚠️ Una sola capa concentra >{porcentaje_pesada:.0f}% de parámetros")
    else:
        print(f"     ✅ Parámetros bien distribuidos")

# Analizar bloques residuales
canales_por_layer = {}
for block in residual_blocks:
    layer_num = block['name'].split('.')[0]  # 'layer1', 'layer2', etc.
    if layer_num not in canales_por_layer:
        canales_por_layer[layer_num] = block['out_channels']

print(f"   • Evolución de canales: ", end="")
print(" → ".join([f"{canales_por_layer[f'layer{i}']}" for i in range(1, 5)]))
print(f"   ✅ Patrón típico de ResNet (duplicación progresiva)")

# ============================================================================
# SECCIÓN 3: RECOMENDACIONES Y PRÓXIMOS PASOS
# ============================================================================

print("\n" + "="*80)
print("🎯 RECOMENDACIONES Y PRÓXIMOS PASOS:")
print("="*80)

print(f"\n📌 BASADO EN EL ANÁLISIS:")

# Recomendación 1: Accuracy
if accuracy < 15:
    print(f"""
1. ⚡ PRIORIDAD ALTA: Fine-tuning del modelo
   • El accuracy ({accuracy:.2f}%) está al nivel del azar
   • Entrenar la última capa mejorará dramáticamente el rendimiento
   • Target esperado: >70% accuracy en CIFAR-10
""")
elif accuracy < 50:
    print(f"""
1. ⚡ PRIORIDAD MEDIA: Continuar entrenamiento
   • El accuracy ({accuracy:.2f}%) muestra algo de aprendizaje
   • Fine-tuning completo puede alcanzar >80%
""")

# Recomendación 2: Activaciones
if len(problemas_detectados) > 0:
    print(f"""
2. 🔧 Revisar capas con problemas:
   • {len(problemas_detectados)} capa(s) muestran comportamiento anómalo
   • Considerar análisis detallado en Notebook 02
""")
else:
    print(f"""
2. ✅ Activaciones saludables
   • Todas las capas funcionan correctamente
   • Listo para análisis profundo de activaciones
""")

# Recomendación 3: Filtros
if tipos_filtros['Inactivos'] > 5:
    print(f"""
3. 🎨 Filtros inactivos detectados:
   • {tipos_filtros['Inactivos']} filtros con baja activación
   • Investigar en Notebook 02: ¿Son útiles para otras imágenes?
""")

# Próximos pasos
print(f"""
📘 PRÓXIMOS PASOS (Notebook 02):

1. 📊 Análisis de Activaciones a Nivel de Dataset
   • Calcular estadísticas sobre TODO el test set (no solo 1 batch)
   • Identificar neuronas "dead" y "superactivas"
   • Analizar patrones de activación por clase

2. 🔍 Análisis Detallado de Capas Profundas
   • Estudiar layer2, layer3, layer4 (no solo conv1)
   • Ver cómo evolucionan las representaciones
   • Comparar activaciones entre capas residuales

3. 📈 Comparación Entre Clases
   • ¿Las activaciones difieren entre gatos y perros?
   • ¿Qué filtros son más importantes para cada clase?
   • Identificar features discriminativas

4. 🎯 Preparación para Feature Visualization (Notebook 03)
   • Seleccionar neuronas interesantes para visualizar
   • Identificar filtros candidatos para activation maximization
   • Diseñar experimentos de interpretabilidad

📁 DATOS GUARDADOS Y DISPONIBLES:
   • Modelo cargado y configurado: ✓
   • Sistema de hooks funcional: ✓
   • Activaciones de ejemplo capturadas: ✓
   • Baseline de rendimiento establecido: ✓
""")

# ============================================================================
# SECCIÓN 4: RESUMEN EJECUTIVO
# ============================================================================

print("="*80)
print("📋 RESUMEN EJECUTIVO:")
print("="*80)

print(f"""
🎯 MODELO: {arch_info['model_name'].upper()}
   • {arch_info['total_params']:,} parámetros ({arch_info['model_size_mb']:.1f} MB)
   • {len(residual_blocks)} bloques residuales
   • Adaptado para {dataset_info['num_classes']} clases (CIFAR-10)

📊 RENDIMIENTO ACTUAL:
   • Accuracy: {accuracy:.2f}% (baseline sin entrenamiento)
   • Confianza: {conf_mean:.2%} promedio
   • Estado: {'⚠️ Requiere entrenamiento' if accuracy < 15 else '✅ Funcional'}

🔬 SALUD DEL MODELO:
   • Capas activas: {capas_saludables}/{len(activation_stats)}
   • Filtros funcionales: {filtros_activos}/{total_filtros}
   • Problemas detectados: {len(problemas_detectados)}

✅ LISTO PARA: Notebook 02 - Análisis de Activaciones por Dataset
""")

print("="*80)
print("✅ NOTEBOOK 01 COMPLETADO")
print("="*80)